In [1]:
from ase.build import root_surface,surface
from ase.io.cif import read_cif
from ase import Atoms
from ase.build import surface, bulk, fcc111
from ase.db import connect
from ase.build import add_adsorbate
from ase.constraints import FixAtoms
from pymatgen.core.structure import Structure
from pymatgen.core.surface import SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.surface import SlabGenerator
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.analysis.local_env import VoronoiNN
from ase.build import molecule
from ase.db import connect
from pymatgen.core.structure import Molecule
from ase.calculators.vasp import Vasp
from ase.io.vasp import write_vasp
from ase.geometry import get_layers

In [ ]:
import os
import shutil
site_name = ['ontop', 'bridge','fcc','hcp']
symbol = ['Cu','Al']
clean_slab = connect('clean_slab.db')
add_slab = connect('add_slab.db')
for sym in symbol:
    cu111 = fcc111(sym,(3,3,6), vacuum=15, periodic=True)
    clean_slab.write(cu111)
    for site in site_name: 
        a = add_adsorbate(cu111,'Na',1.5,'ontop')
        cu111.center()
        layer = get_layers(cu111,(0,0,1),tolerance=0.1)[0]
        index = [j for j in range(len(cu111)) if layer[j] in range(4)]
        constrain = FixAtoms(indices=index)
        cu111.set_constraint(constrain)
        add_slab.write(cu111)
        cwd = os.getcwd()
        dirname = "{}POSCAR_{}".format(sym,site)
        path = os.path.join(cwd,dirname)
        calc = Vasp(prec = 'Accurate',
            directory=path,
            xc = 'PBE',
            kpts = [3,3,1],
            gamma = True,
            lreal = False,
            isif = 2,
            ispin = 2,
            ibrion = 2,
            potim = 0.2,
            ediff = 0.00001,
            ismear = 1,
            sigma = 0.2,
            nsw = 500,
            nelm = 60,
            encut = 500)
        cu111.calc = calc
        cu111.get_potential_energy()
        add_slab.write(cu111, name = dirname)